In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

In [1]:
import os
import math
import matplotlib.pyplot as plt
from google.colab.patches import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pytesseract
import re

IMG_FOLDER_PATH = "/content/drive/MyDrive/DIP_Assignment4"

In [42]:
lMain = "Important Dates (Green)\n\n26/07 to 27/07: Registration/Reporting\n\n28/07: First Day of Classes\n\n04/08: Add/Drop, Late Regstn.\n\n15/03: Withdrawal of course/proj with “‘W’ Grade\n16/1: Last Day of Classes\n\n17-18/11: Preparation Day for End Exams\n\n28/11 to 4/12 Paper Checking & Distribution\n06/12: Start of Winter Vacation"

In [43]:
print(lMain)

Important Dates (Green)

26/07 to 27/07: Registration/Reporting

28/07: First Day of Classes

04/08: Add/Drop, Late Regstn.

15/03: Withdrawal of course/proj with “‘W’ Grade
16/1: Last Day of Classes

17-18/11: Preparation Day for End Exams

28/11 to 4/12 Paper Checking & Distribution
06/12: Start of Winter Vacation


### Detecting dates and events

In [34]:
l0 = "Important Dates (Green)"
l1 = "26/07 to 27/07: Registration/Reporting"
l2 = "28/07: First Day of Classes"
l3 = "04/08: Add/Drop, Late Regstn."
l4 = "15/03: Withdrawal of course/proj with “‘W’ Grade"
l5 = "16/1: Last Day of Classes"
l6 = "17-18/11: Preparation Day for End Exams"
l7 = "28/11 to 4/12 Paper Checking & Distribution"
l8 = "06/12: Start of Winter Vacation"
l9 = "02/12/99: Random event"
l10 = "12/5/21 to 16/6/22: Break"
l11 = "BTP 1st Report(in PPT) Submission: 25/1"
l12 = "Project Grade Due:25/5"
l13 = "27/4,28/4:Lab Exams and"
l14 = "27/4 & 28/4:Lab Exams and"

lMain = [l0, l1, l2, l3, l4, l5, l6, l7, l8]
# 11/02-19/03:
# 17/03-19
# Cases: -
# 

### code not inside function

In [33]:
li = re.split(r" / | /|/ |/| to | to|to |to| - | -|- |-| : | :|: |:| , | ,|, |,| & | &|& |&| ", l14)
# li = re.split(r"/| to | - |-|: |:| ", l12)
# split delimiters: "/", " to ", " - ", "-", ": ", " "
## "&" cannot be delimiter as it doesn't signify an interval, rather two dates
#  having the same event
## although in hindsight it works as such because "&" is used whenever the event
#  is two days long (consecutive days)
date1, month1, year1 = None, None, None
date2, month2, year2 = None, None, None
slCount = 0
clCount = 0
hyCount = 0
idCount = 0
numCount = 0
ids = []

if (li[0].isnumeric()):
    for i in range(len(li)):
        if (li[i].isnumeric()):
            numCount += 1
            ids.append(i)
        
        if not((li[i+1].isnumeric())):
            break
else:
    for i in range(len(li)):
        if (li[i].isnumeric()):
            numCount += 1
            ids.append(i)


if numCount == 2:
    date1, date2 = int(li[ids[0]]), int(li[ids[0]])
    month1, month2 = int(li[ids[1]]), int(li[ids[1]])
    year1, year2 = 2022, 2022 # dynamic value will be obtained by Rishabh
    
# if numCount==3 but third number is >12, then format is DD/MM/YY
elif numCount == 3:
    if int(li[ids[2]]) > 12:
        if int(li[ids[2]]) < 100:
            date1, date2 = int(li[ids[0]]), int(li[ids[0]])
            month1, month2 = int(li[ids[1]]), int(li[ids[1]])
            year1, year2 = 2000+int(li[ids[2]]), 2000+int(li[ids[2]])
        else:
            date1, date2 = int(li[ids[0]]), int(li[ids[0]])
            month1, month2 = int(li[ids[1]]), int(li[ids[1]])
            year1, year2 = int(li[ids[2]]), int(li[ids[2]])
    else:
        date1 = int(li[ids[0]])
        date2 = int(li[ids[1]])
        month1, month2 = int(li[ids[2]]), int(li[ids[2]])
        year1, year2 = 2022, 2022
elif numCount == 4:
    date1, month1 = int(li[ids[0]]), int(li[ids[1]])
    date2, month2 = int(li[ids[2]]), int(li[ids[3]])
    year1, year2 = 2022, 2022
elif numCount == 6:
    if int(li[ids[2]]) < 100:
        date1, month1, year1 = int(li[ids[0]]), int(li[ids[1]]), 2000+int(li[ids[2]])
        date2, month2, year2 = int(li[ids[3]]), int(li[ids[4]]), 2000+int(li[ids[5]])
    else:
        date1, month1, year1 = int(li[ids[0]]), int(li[ids[1]]), int(li[ids[2]])
        date2, month2, year2 = int(li[ids[3]]), int(li[ids[4]]), int(li[ids[5]])

# if numCount==6 (2020-21 Almanac UG1 Term2), then we have 2 dates both of the
# format DD/MM/YY(YY)

print(f"Date1: {date1}-{month1}-{year1}\nDate2: {date2}-{month2}-{year2}")
# print(date1, date2, month1, month2, year1, year2)



Date1: 27-4-2022
Date2: 28-4-2022


### Text to Calendar code inside function

In [ ]:
def txt2Cal(boxDates, year):

    for m in range(len(boxDates)):
        date1, month1, year1 = None, None, None
        date2, month2, year2 = None, None, None
        numCount = 0
        # maxDate = None
        maxID = None
        strList = ""
        ids = []
        li = re.split(r" / | /|/ |/| to | to|to |to| - | -|- |-| : | :|: |:| , | ,|, |,| & | &|& |&| ", boxDates[m])

        # no. of numbers indicating date are counted
        if (li[0].isnumeric()):
            for i in range(len(li)):
                if (li[i].isnumeric()):
                    numCount += 1
                    ids.append(i)
                
                if not((li[i+1].isnumeric())):
                    maxID = i
                    break
        else:
            for i in range(len(li)):
                if (li[i].isnumeric()):
                    numCount += 1
                    ids.append(i)
                    if not(li[i-1].isnumeric()):
                        maxID = i

        # based on no. of numbers obtained, input date format is decided

        # order: date1, month1
        # here date2=date2, month2=month1; year is input year
        if (numCount == 2):
            date1, date2 = int(li[ids[0]]), int(li[ids[0]])
            month1, month2 = int(li[ids[1]]), int(li[ids[1]])
            year1, year2 = year, year

        #3 date numbers are given
        elif (numCount == 3):
            if int(li[ids[2]]) > 12:
                # DD/MM/YY format provided; YY converted to YYYY
                if int(li[ids[2]]) < 100:
                    date1, date2 = int(li[ids[0]]), int(li[ids[0]])
                    month1, month2 = int(li[ids[1]]), int(li[ids[1]])
                    year1, year2 = 2000+int(li[ids[2]]), 2000+int(li[ids[2]])
                # DD/MM/YYYY format provided
                else:
                    date1, date2 = int(li[ids[0]]), int(li[ids[0]])
                    month1, month2 = int(li[ids[1]]), int(li[ids[1]])
                    year1, year2 = int(li[ids[2]]), int(li[ids[2]])
            # order: date1, date2, month1
            # here month2=month1; year is input year
            else:
                date1 = int(li[ids[0]])
                date2 = int(li[ids[1]])
                month1, month2 = int(li[ids[2]]), int(li[ids[2]])
                year1, year2 = year, year
        
        # order: date1, month1, date2, month2
        # year is input year
        elif (numCount == 4):
            date1, month1 = int(li[ids[0]]), int(li[ids[1]])
            date2, month2 = int(li[ids[2]]), int(li[ids[3]])
            year1, year2 = year, year
        
        # order: date1, month1, year1, date2, month2, year2
        elif (numCount == 6):
            if int(li[ids[2]]) < 100:
                date1, month1, year1 = int(li[ids[0]]), int(li[ids[1]]), 2000+int(li[ids[2]])
                date2, month2, year2 = int(li[ids[3]]), int(li[ids[4]]), 2000+int(li[ids[5]])
            else:
                date1, month1, year1 = int(li[ids[0]]), int(li[ids[1]]), int(li[ids[2]])
                date2, month2, year2 = int(li[ids[3]]), int(li[ids[4]]), int(li[ids[5]])
        
        if (li[0].isnumeric()):
            for n in range(maxID+1, len(li)):
                strList += li[n]
        elif not(li[0].isnumeric()):
            for n in range(maxID):
                strList += li[n]
        
        return 

        

In [55]:
ab = "lajksdjnjkashbdibhasd\nashjdgkyuahjgsd"
a = np.array([[2,3],[4,5]]).astype(np.int64)
print(a)

[[2 3]
 [4 5]]
